In [89]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt

In [90]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [91]:
transform = transforms.Compose([
    #transforms.Resize((224, 224)),  # Resize images to a consistent size
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize image data
                         std=[0.229, 0.224, 0.225])
])

In [92]:
dataset = datasets.ImageFolder('D:/Projects/Python Projects/DL/PyTorch/Facial_Expression_Dataset/MMAFEDB/train/', transform=transform)

In [93]:
dt = datasets.ImageFolder('D:/Projects/Python Projects/DL/PyTorch/Facial_Expression_Dataset/MMAFEDB/test/', transform=transform)
dtt = DataLoader(dt, batch_size=32, shuffle=True)

dtt

In [ ]:
valid = datasets.ImageFolder('D:/Projects/Python Projects/DL/PyTorch/Facial_Expression_Dataset/MMAFEDB/test/', transform=transform)
validtd = DataLoader(dt, batch_size=32, shuffle=True)

In [94]:
dataset

Dataset ImageFolder
    Number of datapoints: 92968
    Root location: D:/Projects/Python Projects/DL/PyTorch/Facial_Expression_Dataset/MMAFEDB/train/
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [95]:
dataset.classes

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [96]:
dataset.class_to_idx

{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [97]:
for i in range(7):
    print(dataset.targets.count(i))

6566
3231
4859
28592
29384
12223
8113


In [98]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [99]:
dataloader

In [100]:
for batch in dtt:
    images, labels = batch
    break
images.shape

torch.Size([32, 3, 48, 48])

In [101]:
class Facial_Expression_ModelV1(nn.Module):
    def __init__(self, input_features, hidden, output_features, n):
        super().__init__()
        self.Stack_1 = nn.Sequential(
            # nn.Flatten(start_dim=2),
            nn.Conv2d(input_features, hidden, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(hidden, hidden, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 1, 0)
        )
        self.Stack_2 = nn.Sequential(
            nn.Conv2d(hidden, hidden, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(hidden, hidden, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 1, 0),
            nn.Conv2d(hidden, output_features, 3, 1, 1)
        )
        self.Linearize = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=n, out_features=7)
        )

    def forward(self, X):
        X = self.Stack_1(X)
        X = self.Stack_2(X)
        X = self.Linearize(X)
        return X

In [102]:
model_1 = Facial_Expression_ModelV1(3, 120, 7, 7*46*46)

model_1

Facial_Expression_ModelV1(
  (Stack_1): Sequential(
    (0): Conv2d(3, 120, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(120, 120, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (Stack_2): Sequential(
    (0): Conv2d(120, 120, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(120, 120, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(120, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (Linearize): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=14812, out_features=7, bias=True)
  )
)

In [103]:
images.shape

torch.Size([32, 3, 48, 48])

In [104]:
nn.Flatten()(images).shape

torch.Size([32, 6912])

In [105]:
A = model_1(images)
A = torch.softmax(A, dim=1).argmax(dim=1)
A==labels

tensor([ True, False, False,  True, False, False, False, False, False, False,
        False,  True, False,  True, False, False,  True,  True, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False,  True])

In [106]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_1.parameters(), lr=0.05)

In [107]:
print(len(dataloader))

2906


In [108]:
def train_loop(X, y, Batch):
    model_1.train()
    preds = model_1(X)
    loss = loss_fn(preds, y)
    y_preds = torch.softmax(preds, dim=1).argmax(dim=1)
    accuracy = ((y_preds == y).sum().item() / len(y)) * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if Batch%100==0:
        print(f"Training {Batch}... Loss:{loss:.4f}, accuracy:{accuracy}")
    return (accuracy, loss)

In [109]:
def test_loop(X, y, Batch):
    model_1.eval()
    with torch.inference_mode():
        preds = model_1(X)
        loss = loss_fn(preds, y)
        y_preds  =torch.softmax(preds, dim=1).argmax(dim=1)
        accuracy = ((y_preds == y).sum().item() / len(y)) * 100

    if Batch%100==0:
        print(f"Testing {Batch}... Loss:{loss:.4f}, accuracy:{accuracy}")
    return accuracy, loss

In [110]:
epochs = 10
for epoch in range(epochs):
    print(f"Epoch no.:{epoch}")
    Train_loss, Train_acc = 0., 0.
    Validation_loss, Validation_acc = 0., 0.
    for batch, (X,y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        a, l = train_loop(X, y, batch)
        Train_acc += a
        Train_loss += l
    Train_acc /= len(dataloader)
    Train_loss /= len(dataloader)
    for batch, (X, y) in enumerate(validt):
        X = X.to(device)
        y = y.to(device)
        ta, tl = test_loop(X, y, batch)
        Validation_acc += ta
        Validation_loss += tl
    Validation_loss /= len(validt)
    Validation_acc /= len(validt)

    print(f"Average Metrics...\nTrain_loss:{Train_loss:.4f} | Train_acc:{Train_acc:.2f} | Validate_loss:{Validation_loss:.4f} | Validate_acc:{Validation_acc:.2f}")3

Epoch no.:0
Training 0... Loss:1.9442, accuracy:40.625
Training 100... Loss:1.5082, accuracy:43.75
Training 200... Loss:2.0655, accuracy:15.625
Training 300... Loss:1.7435, accuracy:28.125
Training 400... Loss:2.3574, accuracy:21.875
Training 500... Loss:1.5224, accuracy:37.5
Training 600... Loss:1.5468, accuracy:37.5
Training 700... Loss:1.4046, accuracy:50.0
Training 800... Loss:1.2951, accuracy:62.5
Training 900... Loss:1.4906, accuracy:43.75
Training 1000... Loss:1.2640, accuracy:56.25
Training 1100... Loss:1.4033, accuracy:53.125
Training 1200... Loss:1.2890, accuracy:46.875
Training 1300... Loss:1.4171, accuracy:43.75
Training 1400... Loss:1.3500, accuracy:53.125
Training 1500... Loss:1.4745, accuracy:43.75
Training 1600... Loss:1.1927, accuracy:68.75
Training 1700... Loss:1.1146, accuracy:62.5
Training 1800... Loss:1.1712, accuracy:59.375
Training 1900... Loss:1.1078, accuracy:56.25
Training 2000... Loss:1.1135, accuracy:62.5
Training 2100... Loss:0.9674, accuracy:71.875
Trainin

NameError: name 'validt' is not defined

In [ ]:
for batch, (X, y) in enumerate(dtt):
    Test_acc, Test_loss = 0., 0.
    
    X = X.to(device)
    y = y.to(device)
    ta, tl = test_loop(X, y, batch)
    Test_acc += ta
    Test_loss += tl
    Test_loss /= len(dtt)
    Test_acc /= len(dtt)

print(f"Testing... Loss:{Test_loss:.4f}, accuracy:{Test_acc:.2f}")